#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# clone YOLOv5 repository
%cd /content
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 9473, done.
remote: Total 9473 (delta 0), reused 0 (delta 0), pack-reused 9473
Receiving objects: 100% (9473/9473), 9.91 MiB | 23.70 MiB/s, done.
Resolving deltas: 100% (6584/6584), done.
/content/yolov5
HEAD is now at 886f1c0 DDP after autoanchor reorder (#2421)


In [ ]:
import os
import numpy as np
import shutil

import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

from IPython.core.magic import register_line_cell_magic

!pip install clodsa
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import xml.etree.ElementTree as ET
import cv2
%matplotlib inline

     |████████████████████████████████| 5.7 MB 7.1 MB/s 
     |████████████████████████████████| 276 kB 44.7 MB/s 
  Created wheel for clodsa: filename=clodsa-1.2.44-py2.py3-none-any.whl size=72166 sha256=77359deb8eefe5ee6807f4abf171dc464a38e4a503f580ad6d1cecfa429c6b6e
  Stored in directory: /root/.cache/pip/wheels/0c/02/e3/6c7475202a513df8144bc02e47b079fe6fd44d1b1a73ea4dad
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12093 sha256=baa23f60e5fe11303d941c4fe07373f9c7e6c3b7eaca82929a92ad8ef54af05e
  Stored in directory: /root/.cache/pip/wheels/eb/bb/07/25a7f0718ee3fe137384011b8e56070f91cf950ee6047c287f
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62527 sha256=bb2fe7808094cd39f258b7266927b940710e91860060a4aea2d723897260d768
  Stored in directory: /root/.cache/pip/wheels/92/e3/af/1dc0fdca93232d700ac176af6554cf22b85f3d7e8aeee5ac08
Successfully built clodsa commentjson lark-parser


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 636 kB 5.3 MB/s 
Setup complete. Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


#Gera Data Augmentation

In [ ]:
def create_folder_with_images_and_labels(input_path):
  %rm -rf '/content/imagens_e_anotacoes'
  os.mkdir("/content/imagens_e_anotacoes")

  %cp $input_path$"images"/* /content/imagens_e_anotacoes
  %cp $input_path$"labels"/* /content/imagens_e_anotacoes
  # for image_name,label_name in zip(os.listdir(input_path+"images"),os.listdir(input_path+"labels")):
  #   image_path = os.path.join(input_path+"images", image_name)
  #   label_path = os.path.join(input_path+"labels", label_name)
  #   shutil.copy(image_path, "/content/imagens_e_anotacoes")
  #   shutil.copy(label_path, "/content/imagens_e_anotacoes")


def define_augmentor(OUTPUT_PATH):
  PROBLEM = "detection"
  ANNOTATION_MODE = "yolo"
  INPUT_PATH = "/content/imagens_e_anotacoes"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "yolo"
  
  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,
                              OUTPUT_MODE,
                              GENERATION_MODE,
                              INPUT_PATH,
                              {"outputPath":OUTPUT_PATH})
  transformer = transformerGenerator(PROBLEM)
  return augmentor, transformer

def generate_augmented_set(input_path, dst_path="/content/train/", append_dataset=False):
  create_folder_with_images_and_labels(input_path)
  augmentor, transformer = define_augmentor("/content/augmented_images_yolo")
  
  #Manter Original
  none = createTechnique("none",{})
  augmentor.addTransformer(transformer(none))
  #Rotações
  for i in range(36):
    rotate = createTechnique("rotate", {"angle" : 30 + i*300/35})
    augmentor.addTransformer(transformer(rotate))
  #Flip Ver.
  vFlip = createTechnique("flip",{"flip":0})
  augmentor.addTransformer(transformer(vFlip))
  #Flip Hor.
  hFlip = createTechnique("flip",{"flip":1})
  augmentor.addTransformer(transformer(hFlip))
  #Flip HV
  hvFlip = createTechnique("flip",{"flip":-1})
  augmentor.addTransformer(transformer(hvFlip))
  # # #Blur
  # avgBlur =  createTechnique("average_blurring", {"kernel" : 11})
  # augmentor.addTransformer(transformer(avgBlur))
  # %rm -rf "/content/augmented_images_yolo"
  
  %rm -rf "/content/augmented_images_yolo"
  augmentor.applyAugmentation()

  %rm -rf $dst_path
  os.mkdir(dst_path)
  os.mkdir(dst_path + 'images')
  os.mkdir(dst_path + 'labels')  

  %cp /content/augmented_images_yolo/*.jpg $dst_path$"images"
  %cp /content/augmented_images_yolo/*.txt $dst_path$"labels"

  if append_dataset:
    for dataset in append_dataset:
      for image_name,label_name in zip(os.listdir(dataset+"images"),os.listdir(dataset+"labels")):
        image_path = os.path.join(dataset+"images", image_name)
        label_path = os.path.join(dataset+"labels", label_name)
        shutil.copy(image_path, dst_path + 'images')
        shutil.copy(label_path, dst_path + 'labels')

In [ ]:
# input_path: path to dataset to be augmented
# append_dataset: path to other datasets to be joined. Default=False

#returns: augmented images + joined datasets, if exists. Stored in /content/train
# can take some minutes to run.

generate_augmented_set(input_path="/content/gdrive/MyDrive/Nucleos/Datasets/original/train/",
                       append_dataset=["/content/gdrive/MyDrive/Nucleos/Datasets/brightness_preproc/train/",
                        "/content/gdrive/MyDrive/Nucleos/Datasets/sequential_preproc/train/"])

In [ ]:
print("Número de imagens: ")
!ls -1 /content/gdrive/MyDrive/Nucleos/Datasets/completo/train/images/*.jpg | wc -l
print("Número de anotações: ")
!ls -1 /content/gdrive/MyDrive/Nucleos/Datasets/completo/train/labels/*.txt | wc -l

Número de imagens: 
3036
Número de anotações: 
3036


#Cria splits para validação k-fold

In [ ]:
#@title Gera N Splits


def create_n_folds(images_path, N=10):

  files = os.listdir(images_path)
  n = np.arange(len(files))
  np.random.shuffle(n)
  n_split = np.array_split(n,N)
  folds = np.empty(N,).tolist()
  for i in range(N):
    folds[i] = [files[j] for j in n_split[i]]
  return folds


def make_folders(N=10, dst="/content/cross_val"):
  %rm -rf $dst
  os.mkdir(dst)
  for i in range(N):
    folder = dst + "/split" + str(i+1)
    os.mkdir(folder)
    os.mkdir(folder + "/train")
    os.mkdir(folder + "/test")
    os.mkdir(folder + "/train/images")
    os.mkdir(folder + "/train/labels")
    os.mkdir(folder + "/test/images")
    os.mkdir(folder + "/test/labels")


def copy_image_by_image(ROOT, folds, i, j, idx=True, dst="/content/cross_val"):
  folder = ["/test", "/train"]
  dest_folder = dst + "/split" + str(i+1) + folder[idx]
  
  for image_name in folds[j]:
    image_path = ROOT + image_name
    label_path = ROOT.split("/images")[0] + "/labels/" + image_name.split(".jpg")[0] + ".txt"
    shutil.copy(image_path, dest_folder+"/images")
    shutil.copy(label_path, dest_folder+"/labels")


def create_n_splits(images_path, N=10, dst="/content/cross_val"):
  folds = create_n_folds(images_path, N)
  make_folders(N, dst=dst)
  for i in range(N):
    for j in range(N):
      if i == j:
        copy_image_by_image(images_path, folds, i, j, idx=False, dst=dst)
      else:
        copy_image_by_image(images_path, folds, i, j, dst=dst)

In [ ]:
%%time
%cd ../
N = 5
#path to images
dataset_dir = "/content/train/images/"
create_n_splits(dataset_dir, N)

/
CPU times: user 1.56 s, sys: 5.4 s, total: 6.95 s
Wall time: 7.7 s


##Checagem por desencargo de consciência

In [ ]:
splits_path = "/content/gdrive/MyDrive/Nucleos/Datasets/completo/cross_val/"

for i in range(N):
  print("Split", i, " =====> Imagens de treino:",len(os.listdir(splits_path + "split"+str(i+1)+"/train/images")),
        "Imagens de teste:",len(os.listdir(splits_path + "split"+str(i+1)+"/test/images")))

repeated = []

for i in range(1, 6):
  splitfolder = splits_path + "split" +str(i)
  train = os.listdir(splitfolder + "/train/images")
  test = os.listdir(splitfolder + "/test/images")
  [repeated.append(x) for x in train if x in test]
  if any(repeated):
    print("Repeated imagens on split: ", i)
if not any(repeated):
  print("None repeated image in splits.")

Split 0  =====> Imagens de treino: 2428 Imagens de teste: 608
Split 1  =====> Imagens de treino: 2429 Imagens de teste: 607
Split 2  =====> Imagens de treino: 2429 Imagens de teste: 607
Split 3  =====> Imagens de treino: 2429 Imagens de teste: 607
Split 4  =====> Imagens de treino: 2429 Imagens de teste: 607
None repeated image in splits.


#Inicia Treinamento

In [ ]:
!nvidia-smi

Tue Oct  5 15:11:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
isDataAug = True
cfg = ["original", "completo"]
path = "/content/gdrive/MyDrive/Nucleos/tmp/data_yaml/"
folder = path + cfg[isDataAug]
for file_name in os.listdir(folder):
  file_path = os.path.join(folder, file_name)
  shutil.copy(file_path, "/content/")

In [ ]:
PRE_TRAIN = [True, False]
DEST_DIR_SPLITS = "/content/gdrive/MyDrive/Nucleos/CVal/"
DEST_DIR_RETRAIN = "/content/gdrive/MyDrive/Nucleos/Retreino/"
versions_list = ["s", "m", "l", "x"]


def check_train_config(PT_WEIGHTS, version):
  weight = None
  name = "v5" + version
  mdl = "./models/yolov5" + version + ".yaml"
  if PT_WEIGHTS:
    name = name + "w"
    weight = "yolov5" + version + ".pt"
  if isDataAug:
    name = name + "d"

  return name, weight, mdl


def check_lines_in_results(filename):
  if os.path.exists(filename):
    file = open(filename, "r")
    line_count = 0
    for line in file:
        if line != "\n":
            line_count += 1

    file.close()
    if line_count==200:
      return True
  
  return False


def train_splits(n_splits):
  for PT_WEIGHTS in PRE_TRAIN:
    for version in versions_list:
      name, weight, mdl = check_train_config(PT_WEIGHTS, version)
      
      for i in range(1, n_splits+1):
        name_s = name + str(i)
        data_s = "../split" + str(i) + ".yaml"
        if os.path.isdir(DEST_DIR_SPLITS + name_s):
          print("Train configuration already stored. Available on: ",DEST_DIR_SPLITS + name_s)
          continue
        
        %cd /content/yolov5/
        !python train.py --img 416 --batch 4 --epochs 200 --data $data_s --cfg $mdl --weights $weight --cache  --name $name_s
        
        results_path = "/content/yolov5/runs/train/" + name_s 
        results_txt = os.path.join(results_path, "results.txt")
        if check_lines_in_results(results_txt):
          shutil.copytree(results_path, DEST_DIR_SPLITS + name_s)
      
      name = name + "r"
      if os.path.isdir(DEST_DIR_RETRAIN + name):
          print("Train configuration already stored. Available on: ",DEST_DIR_RETRAIN + name)
          continue
      data_r = "../data.yaml"
      
      %cd /content/yolov5/
      !python train.py --img 416 --batch 4 --epochs 200 --data $data_r --cfg $mdl --weights $weight  --cache --name $name
      
      results_path = "/content/yolov5/runs/train/" + name 
      results_txt = os.path.join(results_path, "results.txt")
      if check_lines_in_results(results_txt):
        shutil.copytree(results_path, DEST_DIR_RETRAIN + name)

In [ ]:
train_splits(n_splits=5)

Train configuration already stored. Available on:  /content/gdrive/MyDrive/Nucleos/CVal/v5swd1
Train configuration already stored. Available on:  /content/gdrive/MyDrive/Nucleos/CVal/v5swd2
/content/yolov5
github: ⚠️ WARNING: code is out of date by 565 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=True, cfg='./models/yolov5s.yaml', data='../split3.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='v5swd3', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/v5swd3', single_cls=False, sync_bn=False, total_batch_size=4, w

In [ ]:
%rm -rf /content/yolov5/runs/train

In [ ]:
src = "/content/gdrive/MyDrive/Podocitos/CVal"
for dir in os.listdir(src):
  dir = os.path.join(src, dir)
  filename = os.path.join(dir, "weights")
  if os.path.exists(filename):
    print(filename, " removed.")
    shutil.rmtree(filename)